In [1]:
from pathlib import Path
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn import functional as F

## HyperParameters

In [16]:
batch_size = 64 #examples at once
block_size = 128 # max context
max_iters = 5000
eval_interval = 1
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
eval_iters = 200
training_split = 0.9
n_embd = 48
n_heads = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)

## Load data to run tokenizer

In [17]:
#training_data = Path('/Users/djemec/data/gpt_train_data/tiny_stories/ts_v2_train.txt')
#validation_data = Path('/Users/djemec/data/gpt_train_data/tiny_stories/ts_v2_valid.txt')
shake_data = Path('/Users/djemec/data/gpt_train_data/tiny_shakespeare/input.txt')

with open(shake_data, 'r', encoding='utf-8') as f:
    text = f.read()

In [18]:
## build out unique characters for the test
chars = sorted(list(set(text)))
vocab_size = len(chars)

## Tokenizer

First we'll show an example but then base it off of tiktoken which is BPE

In [19]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
# To get the tokeniser corresponding to a specific model in the OpenAI API:
# enc = tiktoken.encoding_for_model('gpt2')

In [20]:
def get_batch(split):
    # generate small batches of data of input X and output Y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def generate_n_ex(m, max_new_tokens):
    # start with 0,0 which is initial character and generate
    context = torch.zeros((1,1), dtype=torch.long, device=device)
    generated_tokens = m.generate(context, max_new_tokens=max_new_tokens)
    # decode token to characters
    print(decode(generated_tokens[0].tolist()))

## Model architecture

In [21]:
class Head(nn.Module):
    ''' one head of self attention '''
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B,C,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # decoder block to make sure it only knows the past, not future
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    ''' multiple heads of self attention in parallel'''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out =  torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
        

class FeedForward(nn.Module):
    ''' simple linear layer followed by non-linearity'''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.LeakyReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [22]:
class Block(nn.Module):
    ''' transformer block: communication followed by computation'''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [23]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx) # B,T,C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T, C
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # B, T, vocab_size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # stretch out the array
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

## load data & split

In [24]:
# create split
data = torch.tensor(encode(text), dtype=torch.long)

n = int(training_split*len(data))
train_data = data[:n] 
val_data = data[n:]

In [25]:
# initiate model
model = BigramLanguageModel()
model = model.to(device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

0.181313 M parameters


In [26]:
# better optimizer over stochastic gradient descent
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

### Training with optimmizer

In [27]:
lossi = []

In [29]:
for iter in range(max_iters):
    # periodically evaluate loss
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f'step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}')
        lossi.append(losses['val'])

    #sample data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3812, val loss 4.3844
0
step 1: train loss 4.3247, val loss 4.3293
1


KeyboardInterrupt: 

In [ ]:
plt.plot(lossi)

In [ ]:
generate_n_ex(model, 500)